# import files and directories

In [16]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import uuid
from PIL import Image

In [15]:
POS_PATH = os.path.join('data','positive')
NEG_PATH = os.path.join('data','negetive')
ANC_PATH = os.path.join('data','anchor')

In [4]:
# add directories
os.makedirs(POS_PATH)
os.makedirs(NEG_PATH)
os.makedirs(ANC_PATH)

# Untar Labelled Faces in the Wild Dataset

In [6]:
!wget http://vis-www.cs.umass.edu/lfw/lfw.tgz

--2023-05-13 12:25:11--  http://vis-www.cs.umass.edu/lfw/lfw.tgz
Resolving vis-www.cs.umass.edu (vis-www.cs.umass.edu)... 128.119.244.95
Connecting to vis-www.cs.umass.edu (vis-www.cs.umass.edu)|128.119.244.95|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 180566744 (172M) [application/x-gzip]
Saving to: 'lfw.tgz'

     0K .......... .......... .......... .......... ..........  0% 80.0K 36m43s
    50K .......... .......... .......... .......... ..........  0%  202K 25m38s
   100K .......... .......... .......... .......... ..........  0%  233K 21m17s
   150K .......... .......... .......... .......... ..........  0%  183K 19m58s
   200K .......... .......... .......... .......... ..........  0%  192K 19m2s
   250K .......... .......... .......... .......... ..........  0%  296K 17m30s
   300K .......... .......... .......... .......... ..........  0%  203K 17m4s
   350K .......... .......... .......... .......... ..........  0%  323K 16m3s
   400K .......... .

In [7]:
!tar -xf lfw.tgz

In [8]:
for directory in os.listdir('lfw'):
  for file in os.listdir(os.path.join('lfw',directory)):
    EX_PATH = os.path.join('lfw',directory, file)
    NEW_PATH = os.path.join(NEG_PATH, file)
    os.replace(EX_PATH, NEW_PATH)

In [3]:
# Establish a connection to the webcam
cap = cv2.VideoCapture(0)
while cap.isOpened(): 
    ret, frame = cap.read()
   
    # Cut down frame to 250x250px
    frame = frame[120:120+250,200:200+250, :]
    
    # Collect anchors 
    if cv2.waitKey(1) & 0XFF == ord('a'):
        # Create the unique file path 
        imgname = os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))
        # Write out anchor image
        cv2.imwrite(imgname, frame)
    
    # Collect positives
    if cv2.waitKey(1) & 0XFF == ord('p'):
        # Create the unique file path 
        imgname = os.path.join(POS_PATH, '{}.jpg'.format(uuid.uuid1()))
        # Write out positive image
        cv2.imwrite(imgname, frame)
    
    # Show image back to screen
    cv2.imshow('Image Collection', frame)
    
    # Breaking gracefully
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break
        
# Release the webcam
cap.release()
# Close the image show frame
cv2.destroyAllWindows()

# Data Augmentation

In [51]:
import torchvision.transforms.functional as F
import random
def data_aug(img):
    flag = [0,0,0,0]

    data = []
    while not all(x == 1 for x in flag):
        if flag[0] == 0 and random.randint(0,100) / 100 < 0.5:
            img = F.adjust_brightness(img, brightness_factor=1.1)
            data.append(img)
            flag[0] = 1
        
        if flag[1] == 0 and random.randint(0,100) / 100 < 0.5:
            img = F.adjust_contrast(img, contrast_factor=torch.empty(1).uniform_(0.6, 1).item())
            data.append(img)
            flag[1] = 1
            
        if flag[2] == 0 and random.randint(0,100) / 100 < 0.5:
            img = F.hflip(img)
            data.append(img)
            flag[2] = 1
            
        if flag[3] == 0 and random.randint(0,100) / 100 < 0.5:
            img = F.adjust_saturation(img, saturation_factor=torch.empty(1).uniform_(0.9, 1).item())
            data.append(img)
            flag[3] = 1
        
        
    return data


In [54]:
for file_name in os.listdir(os.path.join(POS_PATH)):
    img_path = os.path.join(POS_PATH, file_name)
    img = Image.open(img_path).convert('RGB')
    img_tensor = F.to_tensor(img)
    augmented_images = data_aug(img_tensor)
    for i, image in enumerate(augmented_images):
        image = F.to_pil_image(image)
        image.save(os.path.join(POS_PATH, '{}.jpg'.format(uuid.uuid1())))

[0, 0, 0, 0]
[0, 1, 0, 0]
[0, 1, 1, 1]
[0, 0, 0, 0]
[1, 1, 0, 0]
[1, 1, 1, 0]
[1, 1, 1, 0]
[0, 0, 0, 0]
[0, 0, 1, 1]
[0, 0, 0, 0]
[1, 1, 0, 0]
[1, 1, 1, 0]
[0, 0, 0, 0]
[1, 0, 0, 1]
[1, 0, 0, 1]
[0, 0, 0, 0]
[0, 1, 0, 1]
[1, 1, 0, 1]
[1, 1, 0, 1]
[0, 0, 0, 0]
[1, 0, 1, 0]
[0, 0, 0, 0]
[1, 0, 0, 1]
[1, 0, 0, 1]
[1, 1, 0, 1]
[0, 0, 0, 0]
[1, 1, 0, 1]
[1, 1, 0, 1]
[1, 1, 0, 1]
[1, 1, 0, 1]
[1, 1, 0, 1]
[1, 1, 0, 1]
[0, 0, 0, 0]
[1, 1, 1, 0]
[1, 1, 1, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 1]
[0, 1, 0, 1]
[0, 1, 0, 1]
[0, 1, 1, 1]
[0, 1, 1, 1]
[0, 0, 0, 0]
[0, 1, 1, 1]
[0, 1, 1, 1]
[0, 0, 0, 0]
[1, 1, 0, 1]
[0, 0, 0, 0]
[1, 0, 0, 0]
[1, 0, 1, 1]
[0, 0, 0, 0]
[1, 1, 1, 0]
[0, 0, 0, 0]
[0, 0, 1, 0]
[1, 0, 1, 0]
[1, 0, 1, 1]
[0, 0, 0, 0]
[1, 0, 0, 0]
[1, 1, 0, 0]
[0, 0, 0, 0]
[0, 1, 1, 0]
[0, 0, 0, 0]
[0, 1, 1, 0]
[0, 0, 0, 0]
[0, 1, 1, 0]
[1, 1, 1, 0]
[1, 1, 1, 0]
[0, 0, 0, 0]
[1, 0, 1, 0]
[1, 1, 1, 0]
[1, 1, 1, 0]
[0, 0, 0, 0]
[0, 1, 0, 0]
[1, 1, 0, 1]
[1, 1, 0, 1]
[1, 1, 0, 1]
[1, 1, 0, 1]